## Environment

In [1]:
import os
from moviepy.editor import VideoFileClip
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from tqdm import tqdm
import librosa
import librosa.display

In [2]:
# Path
data_raw_dir = "/home/tamires/projects/rpp-aevans-ab/tamires/data/derived/"
csv_file = "df_face_times_barbiesmallest10.csv"
movie = "barbiesmallest10.mp4"
output_dir = "barbie_subclips_15/"

## Information

In [4]:
df = pd.read_csv(data_raw_dir+csv_file, index_col=0)
df

,second,type,face_confidence,emotion,emotion_confidence,face_match_prior
1,0.1,multiple faces,NaN,NaN,NaN,NaN
2,0.2,multiple faces,NaN,NaN,NaN,NaN
3,0.3,single face,0.99,happy,1.0,False
4,0.4,single face,0.99,happy,1.0,True
5,0.5,single face,0.99,happy,1.0,True
...,...,...,...,...,...,...
1196,119.6,multiple faces,NaN,NaN,NaN,NaN
1197,119.7,multiple faces,NaN,NaN,NaN,NaN
1198,119.8,multiple faces,NaN,NaN,NaN,NaN
1199,119.9,multiple faces,NaN,NaN,NaN,NaN


#### Distribution of face types and emotion in the data

In [5]:
df.type.value_counts()

type
multiple faces    658
single face       323
no faces          219
Name: count, dtype: int64

In [6]:
df[df.type=="single face"].emotion.value_counts()

emotion
neutral     99
fear        86
happy       55
sad         36
angry       30
surprise    17
Name: count, dtype: int64

#### Filtering the frames with high confidence

In [7]:
df[df.type=="single face"][["face_confidence", "emotion_confidence"]].describe()

,face_confidence,emotion_confidence
count,323.000000,323.000000
mean,0.996223,0.760402
std,0.010335,0.192356
min,0.940000,0.270000
25%,1.000000,0.600000
50%,1.000000,0.780000
75%,1.000000,0.950000
max,1.000000,1.000000


In [8]:
df_filter = df[df.type == "single face"]
df_filter = df_filter[df_filter.emotion_confidence > 0.7]
df_filter = df_filter[df_filter.second > 15]
len(df_filter)/len(df)

0.14083333333333334

In [9]:
df_filter

,second,type,face_confidence,emotion,emotion_confidence,face_match_prior
151,15.1,single face,1.00,sad,0.89,False
155,15.5,single face,0.98,angry,0.83,False
157,15.7,single face,0.98,angry,0.84,False
163,16.3,single face,0.98,fear,0.89,False
166,16.6,single face,0.99,fear,0.99,True
...,...,...,...,...,...,...
1041,104.1,single face,0.99,angry,0.91,True
1043,104.3,single face,0.99,neutral,0.95,True
1047,104.7,single face,0.99,happy,0.99,False
1049,104.9,single face,1.00,happy,1.00,False


#### Resolution difference between movies

In [10]:
VideoFileClip(data_raw_dir+"barbiesmallest10.mp4").size, VideoFileClip(data_raw_dir+"trailer10.mp4").size

([1920, 960], [640, 360])

## Create clips

#### Creating clips: sequential_images.mp4 and audio.mp3

In [13]:
def cut_subclips(row, video_clip, output_dir, lag=60):
    start_time = round(row['second'] - lag, 1)
    end_time = round(row['second'] - 0.1, 1) 
    emotion = row['emotion']
    
    # Write the images file
    images_clip = video_clip.subclip(start_time, end_time).without_audio()
    images_path = os.path.join(output_dir, f"{start_time}_{end_time}_{emotion}_clip_images.mp4")
    images_clip.write_videofile(images_path, fps=10)
    
    # Write the audio file
    audio_clip = video_clip.audio.subclip(start_time, end_time)
    audio_path = os.path.join(output_dir, f"{start_time}_{end_time}_{emotion}_clip_audio.mp3")
    audio_clip.write_audiofile(audio_path)

# Load Movie
video_clip = VideoFileClip(data_raw_dir+movie)

# Set a standard resolution (width, height)
video_clip = video_clip.resize((640, 360)) 

# Create several subclips and mel images
results = df_filter.apply(lambda row: cut_subclips(row, video_clip, data_raw_dir+output_dir, lag=15), axis=1)
video_clip.close()

Moviepy - Building video /home/tamires/projects/rpp-aevans-ab/tamires/data/derived/barbie_subclips_15/0.1_15.0_sad_clip_images.mp4.
Moviepy - Writing video /home/tamires/projects/rpp-aevans-ab/tamires/data/derived/barbie_subclips_15/0.1_15.0_sad_clip_images.mp4



Moviepy - Done !
Moviepy - video ready /home/tamires/projects/rpp-aevans-ab/tamires/data/derived/barbie_subclips_15/0.1_15.0_sad_clip_images.mp4
MoviePy - Writing audio in /home/tamires/projects/rpp-aevans-ab/tamires/data/derived/barbie_subclips_15/0.1_15.0_sad_clip_audio.mp3


MoviePy - Done.
Moviepy - Building video /home/tamires/projects/rpp-aevans-ab/tamires/data/derived/barbie_subclips_15/0.5_15.4_angry_clip_images.mp4.
Moviepy - Writing video /home/tamires/projects/rpp-aevans-ab/tamires/data/derived/barbie_subclips_15/0.5_15.4_angry_clip_images.mp4



Moviepy - Done !
Moviepy - video ready /home/tamires/projects/rpp-aevans-ab/tamires/data/derived/barbie_subclips_15/0.5_15.4_angry_clip_images.mp4
MoviePy - Writing audio in /home/tamires/projects/rpp-aevans-ab/tamires/data/derived/barbie_subclips_15/0.5_15.4_angry_clip_audio.mp3


MoviePy - Done.
Moviepy - Building video /home/tamires/projects/rpp-aevans-ab/tamires/data/derived/barbie_subclips_15/0.7_15.6_angry_clip_images.mp4.
Moviepy - Writing video /home/tamires/projects/rpp-aevans-ab/tamires/data/derived/barbie_subclips_15/0.7_15.6_angry_clip_images.mp4



...

MoviePy - Done.
Moviepy - Building video /home/tamires/projects/rpp-aevans-ab/tamires/data/derived/barbie_subclips_15/89.9_104.8_happy_clip_images.mp4.
Moviepy - Writing video /home/tamires/projects/rpp-aevans-ab/tamires/data/derived/barbie_subclips_15/89.9_104.8_happy_clip_images.mp4



Moviepy - Done !
Moviepy - video ready /home/tamires/projects/rpp-aevans-ab/tamires/data/derived/barbie_subclips_15/89.9_104.8_happy_clip_images.mp4
MoviePy - Writing audio in /home/tamires/projects/rpp-aevans-ab/tamires/data/derived/barbie_subclips_15/89.9_104.8_happy_clip_audio.mp3


MoviePy - Done.
Moviepy - Building video /home/tamires/projects/rpp-aevans-ab/tamires/data/derived/barbie_subclips_15/90.1_105.0_happy_clip_images.mp4.
Moviepy - Writing video /home/tamires/projects/rpp-aevans-ab/tamires/data/derived/barbie_subclips_15/90.1_105.0_happy_clip_images.mp4



Moviepy - Done !
Moviepy - video ready /home/tamires/projects/rpp-aevans-ab/tamires/data/derived/barbie_subclips_15/90.1_105.0_happy_clip_images.mp4
MoviePy - Writing audio in /home/tamires/projects/rpp-aevans-ab/tamires/data/derived/barbie_subclips_15/90.1_105.0_happy_clip_audio.mp3


MoviePy - Done.


#### Converting audio: audio.mp3 to mel.png

In [3]:
# List all mp3 files in the directory
files = os.listdir(data_raw_dir+output_dir)
mp3_files = [file for file in files if file.endswith('.mp3')]

for audio_path in tqdm(mp3_files, desc="Processing MP3 files"):
    output_path = data_raw_dir+output_dir+audio_path.replace('.mp3', '.png')
    #print(output_path)

    # Check if file already exist
    if os.path.isfile(output_path):
        #print(output_path)
        continue
        
    else:
        try:
            # Audio to Mel Spectrogram
            y, sr = librosa.load(data_raw_dir+output_dir+audio_path, sr=None)  # Load audio file: y is the audio time series, sr is the sampling rate
            S = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=128, fmax=8000) # Compute Mel spectrogram
            S_dB = librosa.power_to_db(S, ref=np.max) # Convert to decibels
           
            # Plot Mel spectrogram
            plt.figure(figsize=(30, 12)) 
            librosa.display.specshow(S_dB, sr=sr)
            plt.savefig(output_path)
            plt.close()
        except Exception as e:
            print(f"Error processing {audio_path}: {e}")
            continue

Processing MP3 files: 100%|████████████████████████████████████████████████████████████████████████████████████████████| 169/169 [00:22<00:00,  7.59it/s]
